# ARIMAX model analysis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
preds_and_target = pd.read_csv('data/model_inputs/predictors_and_targets.csv')
preds_and_target.head()

,state_abbr,year,population,violent_crime,homicide,rape,robbery,aggravated_assault,property_crime,burglary,...,burglary_1000,larceny_1000,motor_vehicle_theft_1000,arson_1000,avg_unemployment_rate,avg_CPI,ag_Democrat,ag_Mixed,ag_Republican,ag_Unknown
0,AL,1979,3769000,15578,496,1037,4127,9918,144372,48517,...,12.872645,22.231626,3.200849,0.067392,7.225000,72.583333,1,0,0,0
1,AL,1980,3861466,17320,509,1158,5102,10551,173191,58952,...,15.266741,26.422348,3.162012,0.287197,8.816667,82.383333,1,0,0,0
2,AL,1981,3916000,18423,465,1021,4952,11985,173411,56811,...,14.507406,26.933350,2.841931,0.305158,10.691667,90.933333,1,0,0,0
3,AL,1982,3943000,17653,417,1026,4417,11793,165048,49531,...,12.561755,26.563530,2.733198,0.273396,13.950000,96.533333,1,0,0,0
4,AL,1983,3959000,16471,364,931,3895,11281,145890,42485,...,10.731245,23.813842,2.305128,0.247790,13.816667,99.583333,1,0,0,0


In [3]:
state_abbrs = pd.unique(preds_and_target['state_abbr'])
state_abbrs

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
       'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
       'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC'], dtype=object)

## Determining the best p, d, q

In [4]:
arima_violent = {}
arima_property = {}

for abbr in state_abbrs:
    # Create a new dataframe for each state
    df = preds_and_target.copy()
    df =df[df['state_abbr'] == abbr]
    df['year'] = pd.to_datetime(df['year'], format='%Y')
    df.set_index('year',inplace=True)
    
    # Predictor variables
    X = df[['population', 'avg_unemployment_rate', 'avg_CPI', 'ag_Democrat', 'ag_Mixed', 'ag_Republican']]
    
    # First target variable
    y1 = df['violent_crime_1000']
    arima1 = auto_arima(
        y1,
        X,
        start_p = 0,
        start_q = 0,
        d = 2, # Most data is stationary at 2 differences
        max_d = 2,
        start_P = 0,
        start_Q = 0,
        D = 2,
        max_D = 2,
        seasonal = False,
        suppress_warnings = True,
        trace = False,
        error_action = 'ignore'
    )
    
    arima_violent[abbr] = arima1
    
    # Second target variable
    y2 = df['property_crime_1000']
    arima2 = auto_arima(
        y2,
        X,
        start_p = 0,
        start_q = 0,
        d = 2, # Most data is stationary at 2 differences
        max_d = 2,
        start_P = 0,
        start_Q = 0,
        D = 2,
        max_D = 2,
        seasonal = False,
        suppress_warnings = True,
        trace = False,
        error_action = 'ignore'
    )
    
    arima_property[abbr] = arima2

## ARIMAX analysis

In [13]:
violent_mae_total = 0
violent_r2_total = 0
violent_rmse_total = 0

for abbr in state_abbrs:
    df = preds_and_target.copy()
    df = df[df['state_abbr'] == abbr]
    df['year'] = pd.to_datetime(df['year'], format='%Y')
    df.set_index('year',inplace=True)

    X = df[['population', 'avg_unemployment_rate', 'avg_CPI', 'ag_Democrat', 'ag_Mixed', 'ag_Republican']]
    current_arima = ARIMA(endog = df['violent_crime_1000'], 
                          exog = X,
                          order=arima_violent[abbr].order)
    fitted_arima = current_arima.fit()
    predictions = fitted_arima.predict(start = 1, dynamic = False)
    
    violent_mae_total += mean_absolute_error(df['violent_crime_1000'][1:], predictions)
    violent_r2_total += r2_score(df['violent_crime_1000'][1:], predictions)
    violent_rmse_total += mean_squared_error(df['violent_crime_1000'][1:], predictions)**0.5

In [14]:
avg_violent_mae = violent_mae_total/51
avg_violent_mae

0.36867118654025127

In [15]:
avg_violent_rmse = violent_rmse_total/51
avg_violent_rmse

0.7947434853115708

Our ARIMA models for each state are, on average, off by 0.36 violent crimes per 1000 population using mean absolute error and 0.79 violent crimes per 1000 population using root mean squared error.

In [8]:
avg_violent_r2 = violent_r2_total/51
avg_violent_r2

-0.5313733393689813

Despite the low MAE and RMSE, the presence of a negative R-squared indicates that using an ARIMA model is a poor fit for the violent crime data.

In [9]:
property_mae_total = 0
property_r2_total = 0
property_rmse_total = 0

for abbr in state_abbrs:
    df = preds_and_target.copy()
    df = df[df['state_abbr'] == abbr]
    df['year'] = pd.to_datetime(df['year'], format='%Y')
    df.set_index('year',inplace=True)

    X = df[['population', 'avg_unemployment_rate', 'avg_CPI', 'ag_Democrat', 'ag_Mixed', 'ag_Republican']]
    current_arima = ARIMA(endog = df['property_crime_1000'], 
                          exog = X,
                          order=arima_property[abbr].order)
    fitted_arima = current_arima.fit()
    predictions = fitted_arima.predict(start = 1, dynamic = False)
    property_mae_total += mean_absolute_error(df['property_crime_1000'][1:], predictions)
    property_r2_total += r2_score(df['property_crime_1000'][1:], predictions)
    property_rmse_total += mean_squared_error(df['property_crime_1000'][1:], predictions)**0.5

In [10]:
avg_property_mae = property_mae_total/51
avg_property_mae

2.2508470195804366

In [11]:
avg_property_rmse = property_rmse_total/51
avg_property_rmse

4.790785057962381

Our ARIMA models for each state are, on average, off by 2.24 property crimes per 1000 population using mean absolute error and 4.77 property crimes per 1000 population using root mean squared error.

In [12]:
avg_property_r2 = property_r2_total/51
avg_property_r2

0.5424504398646747

On average, our ARIMA models for each state explain 54.4% of the variation in property crime over time.